In [3]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [4]:
mse = torch.nn.MSELoss()

# Train Model

In [62]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
train_dataset_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)
unlabelled_dataset_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=64, shuffle=True)

In [63]:
print(type(train_dataset_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [65]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False


def loss_function(recon_x, x, mu, logvar):
    mean_squared =mse(X_hat, data)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return mean_squared + KLD


In [66]:
model = VAE() #torch.load("decon-vae-50.p")
model_name = 'sdfdenoise'

optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [67]:
# CPU only training
def generate_train(epoch,  data_loader,isSupervised = False):
    model.train()
    train_loss = 0
    classification_loss = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        #
        data, target = Variable(data), Variable(target)
        
        # Add noise
        if model_name == 'denoise':
            noise = torch.FloatTensor(len(data), 1,28,28)
            noiseV = Variable(noise)
            noiseV.resize_as(data)
            noiseV.data.normal_(0, 1)
            data.data.add_(noiseV.data)
        
        optimizer.zero_grad()
        x_new, mu, logvar = model(data)
        kl_loss = 0.5 * torch.mean(torch.exp(logvar) + mu ** 2 - 1. - logvar)
        recon_loss = mse(x_new, data) + kl_loss
        class_loss = 0
        if isSupervised:
            output = F.log_softmax(mu)
            class_loss = F.nll_loss(output, target)
        loss = recon_loss + class_loss
        loss.backward()
        train_loss += loss.data[0]
        classification_loss += class_loss.data[0]
        optimizer.step()
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, classification_loss / len(data_loader.dataset)))

    
def generate_test(epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        x_new, mu, logvar = model(data)
        output = F.log_softmax(mu) 
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [68]:
# Generative Classifier
for epoch in range(1, 50):
    generate_train(epoch, data_loader=train_dataset_loader, isSupervised = False)
    generate_train(epoch, data_loader=unlabelled_dataset_loader, isSupervised = True)
    generate_test(epoch, valid_loader)

AttributeError: 'int' object has no attribute 'data'

In [537]:
# Save model
torch.save(model, "decon-vae-50.p")


In [247]:
# Check generated images
from  torchvision.utils import save_image
for index, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    gen_data, mu, logvar = model(data)
    gen_images = gen_data.view(-1, 1,28,28)
    #print(data[0])
    save_image(gen_images.data, 'images/gen{}.jpg'.format(index))


In [ ]:
# CPU only training
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.8)
def train(epoch):
    classifier.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        gen_data, mu, logvar = model(data)
        #Check reconstruction loss 
        recon_loss = reconstruction_function(gen_data, data)
        #print("Reconstruction loss is", recon_loss)
        #print("Type of generated data", type(gen_data))
        
        optimizer.zero_grad()
        output = classifier(gen_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    classifier.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        gen_data, mu, logvar = model(data)
        output = classifier(gen_data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    


In [ ]:
#Classification phase
for epoch in range(1, 10):
    train(epoch)
    test(epoch, valid_loader)

# Create Sample Submission

In [ ]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [ ]:
test(1, test_loader)

In [ ]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [ ]:
label_predict

In [ ]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [ ]:
predict_label.head()

In [ ]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
